# Data Extractor

In [1]:
import ee
ee.Initialize()
ee.Authenticate()
import geetools
import geemap
import os

Enter verification code: 4/1ARtbsJoRkgTPPjVQfT-m41enF447KlMgVZ2u9iRuLp1soLUInHipgoAzmYI

Successfully saved authorization token.


In [2]:
# Parameters.
data_collection = "COPERNICUS/S2_SR_HARMONIZED"
# Time windows.
start_date = "2021-08-01"
end_date = "2021-08-20"
# Color bands.
bands =  ['B4','B3','B2'] # RGB channels.
# Coordinates
coordinates = {
    "Germany" : {
        "p1" : [9.368331, 52.409899],
        "p2" : [10.041240, 52.489424],
        "p3" : [10.109178, 52.295949],
        "p4" : [9.575380, 52.244477],
    }
}

In [3]:
# Create a polygon based on a JSON entry.
polygonGeoJSON = ee.Geometry.Polygon(
    [
        # Outside ring -> [lat, long]
        [
            coordinates["Germany"]["p1"],
            coordinates["Germany"]["p2"],
            coordinates["Germany"]["p3"],
            coordinates["Germany"]["p4"],
            coordinates["Germany"]["p1"],
        ],
    ]
)

In [4]:
Map = geemap.Map()
Map.addLayer(polygonGeoJSON, {}, 'polygonGeoJSON')
Map.centerObject(polygonGeoJSON)

In [5]:
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [6]:
image_collection = (
    ee.ImageCollection(data_collection)
    .select(bands)
    .filterBounds(polygonGeoJSON)
    .filterDate(start_date, end_date)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
)

In [7]:
print(f'The size of the collection is: {image_collection.size().getInfo()}')

The size of the collection is: 1


In [8]:
visualization = {
    "bands" : bands,
    "min": 300, 
    "max": 3500
}

Map.addLayer(image_collection, visualization, 'RGB')

In [9]:
for band in bands:
    extra = dict(sat="S-HARMONIZED", band=band)
    geetools.batch.Export.imagecollection.toDrive(
        collection=image_collection.select(band),
        region=polygonGeoJSON,
        namePattern="{sat}_{system_date}_{band}",
        datePattern = "ddMMMy",
        dataType="int",
        folder="ClimateData",
        extra=extra,
        verbose=True
    )

exporting S-HARMONIZED_14Aug2021_B4 to folder 'ClimateData' in GDrive
exporting S-HARMONIZED_14Aug2021_B3 to folder 'ClimateData' in GDrive
exporting S-HARMONIZED_14Aug2021_B2 to folder 'ClimateData' in GDrive
